In [4]:
import numpy as np

In [1]:
sample = """#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#"""

In [10]:
def parse(sample):
    result = [np.array([list(x) for x in p.split()]) for p in sample.split("\n\n")]
    return result

psample = parse(sample)
psample

[array([['#', '.', '#', '#', '.', '.', '#', '#', '.'],
        ['.', '.', '#', '.', '#', '#', '.', '#', '.'],
        ['#', '#', '.', '.', '.', '.', '.', '.', '#'],
        ['#', '#', '.', '.', '.', '.', '.', '.', '#'],
        ['.', '.', '#', '.', '#', '#', '.', '#', '.'],
        ['.', '.', '#', '#', '.', '.', '#', '#', '.'],
        ['#', '.', '#', '.', '#', '#', '.', '#', '.']], dtype='<U1'),
 array([['#', '.', '.', '.', '#', '#', '.', '.', '#'],
        ['#', '.', '.', '.', '.', '#', '.', '.', '#'],
        ['.', '.', '#', '#', '.', '.', '#', '#', '#'],
        ['#', '#', '#', '#', '#', '.', '#', '#', '.'],
        ['#', '#', '#', '#', '#', '.', '#', '#', '.'],
        ['.', '.', '#', '#', '.', '.', '#', '#', '#'],
        ['#', '.', '.', '.', '.', '#', '.', '.', '#']], dtype='<U1')]

In [17]:
psample[0]

array([['#', '.', '#', '#', '.', '.', '#', '#', '.'],
       ['.', '.', '#', '.', '#', '#', '.', '#', '.'],
       ['#', '#', '.', '.', '.', '.', '.', '.', '#'],
       ['#', '#', '.', '.', '.', '.', '.', '.', '#'],
       ['.', '.', '#', '.', '#', '#', '.', '#', '.'],
       ['.', '.', '#', '#', '.', '.', '#', '#', '.'],
       ['#', '.', '#', '.', '#', '#', '.', '#', '.']], dtype='<U1')

In [33]:
np.flip(psample[0][:,5:], 1)

array([['.', '#', '#', '.'],
       ['.', '#', '.', '#'],
       ['#', '.', '.', '.'],
       ['#', '.', '.', '.'],
       ['.', '#', '.', '#'],
       ['.', '#', '#', '.'],
       ['.', '#', '.', '#']], dtype='<U1')

In [34]:
psample[0][:,5:]

array([['.', '#', '#', '.'],
       ['#', '.', '#', '.'],
       ['.', '.', '.', '#'],
       ['.', '.', '.', '#'],
       ['#', '.', '#', '.'],
       ['.', '#', '#', '.'],
       ['#', '.', '#', '.']], dtype='<U1')

In [36]:
psample[0][:,1:5]

array([['.', '#', '#', '.'],
       ['.', '#', '.', '#'],
       ['#', '.', '.', '.'],
       ['#', '.', '.', '.'],
       ['.', '#', '.', '#'],
       ['.', '#', '#', '.'],
       ['.', '#', '.', '#']], dtype='<U1')

In [38]:
(psample[0][:,1:5] == np.flip(psample[0][:,5:], 1)).all()

True

want 0 - 1, 01 23, then 012 345, then 23 45 then 4 5
but 12 34 isn't valid because it doesn't touch an edge (ie includes 0 or 5)
basically start loop over a start index and lengths like
(0, 1), (0, 2), (0, 3)

ah, so I want to do from 0 to len/2 and then from len-len/2


In [210]:
def samples(size):
    mid = size // 2
    f = [((0, x + 1), (x + 1, 2 * (x + 1))) for x in range(0,mid)] # off by one on the end somehow...
    # this bit is wrong somehow
    # I want (len -2, len - 1), (len - 1, len) 1
    # then
    # (len - 4, len - 2), (len - 2, len) 2
    # then (len - 6, len -3), (len - 3, len)
    b = [((size - (2 * x), size - x), (size - x, size)) for x in range(1, mid + 1)]
    return f + b

samples(9) # (3,5) means 2:3, to 4:5 but (0,2) means 0:1 to 2:3

[((0, 1), (1, 2)),
 ((0, 2), (2, 4)),
 ((0, 3), (3, 6)),
 ((0, 4), (4, 8)),
 ((7, 8), (8, 9)),
 ((5, 7), (7, 9)),
 ((3, 6), (6, 9)),
 ((1, 5), (5, 9))]

In [215]:
def find_mirror(pattern):
    for l_range, r_range in samples(pattern.shape[1]):
        
        l_side = pattern[:,l_range[0]:l_range[1]]
        r_side = np.flip(pattern[:, r_range[0]:r_range[1]], 1)
        if (l_side == r_side).all():
            return l_range[1] # number of columns to left of mirror
    return None

find_mirror(psample[1].T)

4

In [219]:
def score(pattern):
    s = find_mirror(pattern)
    if s is not None:
        return s
    else:
        return 100 * find_mirror(pattern.T)

score(psample[0]), score(psample[1])

(5, 400)

In [220]:
sum(score(p) for p in psample)

405

In [221]:
actual = """#..#.##.#..
###.####.##
###.####.##
###.####.##
###.####.##
#..#.##.#..
.#..#..#..#
..#.####.#.
####....###
..#......#.
.#.........
#...#..#...
####....###
.##......##
#...####...
###.#..#.##
#.########.

..#....#.
..######.
..###.##.
####..###
...####..
###.##.##
##.####.#
...#..#..
..######.

.##......##...#
..########....#
##.#....#.####.
.#...##...#..#.
.##.####.##..##
#.###..###.##.#
....#..#.......
.....##........
##..#..#..####.
...#.##.#......
.##########..##
#.##....##.##.#
.#..####..#..#.

####..#.#####
.#.#.#.#.##..
..#.##....#.#
#...#..##....
###.####.#.#.
..#....#.#..#
..#....#.#..#
###.####.#.#.
#...#..##....
..#.##....#.#
.#.#.#.#.###.
####..#.#####
####..#.#####

.##.####.##.#
#..#.##.#..##
...#.##.#....
#..#....#..#.
#..#....#..#.
...#.##.#....
#..#.##.#..##
.##.####.##.#
.##########.#
####....####.
#.#......#.#.
.##.####.##..
..####.###..#

...#..##..#
#....####..
.#.##.##.##
.#.##.##.##
#....####..
...#..##..#
##.##....#.
####.####.#
#.#..####..

...#..##..#..
..#..####..#.
...###..###..
..##.####.##.
..##......##.
....######...
..##.....###.
######..#####
.....#..#....
##.#.#..#.#.#
...#......#..
....##..##...
..#.######.#.
##.#.####.#.#
...##.##.##..
##.#..##..#.#
..####..####.

#.#..##..#.##
.##.#..#.##..
##..####..###
.#..#..#..#..
..#.#....#...
.#.#.##.#.#..
##..####..###

#.#..#.#.#.
##.....#..#
#.#.####..#
...#...###.
##....###..
#.#.#.....#
.##..#..###
.##..#..###
#.#.#.....#
##....###..
...#...###.
#.######..#
##.....#..#
#.#..#.#.#.
#.#..#.#.#.

##.##.#.######.
....##.........
......##.####.#
....####......#
..##.#.#.#..#.#
.###...#.####.#
###.#.##......#
....###..#..#..
...##.##.#..#.#
..##.##.##..##.
..#..#.#.#..#.#
##.#..###....##
..##..#........
..#.#..........
##.#.#.#..##..#
#####..........
....#.....##...

#....##......##..
########.##.#####
#.##.##..##..##.#
#....#.######.#..
.#..#.##.##.##.#.
.#..#.##....##.#.
.####..##..##..##
.#..#.#.####.#.#.
#.##.##...#..##.#
#######.#..#.####
##..###.#..#.###.
#....##.####.##..
.#..#...#..#...#.

....#..##
..#.#..##
..#.#.##.
..#.#..#.
.......#.
#.....#.#
.#.######
.####..#.
.####..#.
.#.######
#.....#.#
.......#.
..#.#..#.
..#.#.##.
..#.#..##
....#..##
.##.###.#

#.##..#..####
..#..###.#.#.
.##.#.#..##.#
#...#..###...
#...#..###...
.##.#.#..##.#
..#..###.#.#.
#.##..#..####
###.#.###..##
.#.#....###.#
##..##....##.
##..##....###
.#.#....###.#
###.#.###..##
#.##..#..####
..#..###.#.#.
.##.#.#..##.#

####.#.........
####.##..#..#..
....##.#..##..#
.....#..#....#.
.##.#...#....#.
########..##..#
#..###...#..#..
.##.......###..
#..###.###..###
#..#.#.#..##..#
....##..##..##.
.##.##.#..##..#
####.#...####..

#.#.######.
.##.#....#.
##.########
.##.#.##.#.
.##.#.##.#.
##.########
.##.#....#.
#.#.######.
...#..##..#
.#..#..#.#.
.##.#....#.

###...##..##...##
##.#..........#.#
###.#.##..##.#..#
..#...######...#.
..##.##....##.##.
####.#.####.#.###
...#.#.#..#.#.#..
..##.##....##.##.
####..##..##..###
...##..#..#..##..
..###...##...###.

##.##....##...#
.####....####..
#####....####..
#...####.#.....
#..##..####...#
#..##......##..
.##...##..#..##
#.#...#.#.#..##
..##.#..####...
..##.#..####...
#.#...#.#.#..##
.##...##..#..##
#..##......##..
#..##..####...#
#...####.#.....
#####....####..
.####....####..

..#########..
#...##...#..#
#...#.#......
.##..##..#...
.##....##...#
..#..##.##..#
..#..##.##..#
.##....##...#
.##..##..#...
#...#.#......
#...##...#..#
..#########..
.#.......####
#.#.##.#....#
....###......
.......##...#
.......##.#.#

#..#.#..####..#.#
###.#.#......#.#.
..####..#..#..###
###.#.#.#..#.#.#.
..##..#......#..#
...#..######.#..#
##.#..#.#..#.#..#
.##....######....
.####...####...##
#######......####
##....##.##.##...
#.####...##...###
#.####...##...###

....#..
###.##.
#.###..
.#.#.##
.#.#.##
#.###..
###.##.
....#.#
.#.#..#
.#.#..#
....#.#

##..######.#.#.
##..######.#.##
..##...#..#.#.#
#....#..###.#..
#.##.#.#.####.#
.#..#.##.#...#.
..##..##.##..#.
#....###...####
#.##.######....
#.##.#.#..#.#.#
##..##.##.#####

.####...#.#..#.
.##.#..##.##..#
.####..##.##..#
.####...#.#..#.
###.#.#.###....
#.#..#.###..###
#.#..##..#.##..
....#..#..#.##.
#.#.#..#.##.#.#
#.#.#..#.##.#.#
....#..#..#.##.
#.#..##..#.##..
#.#..#.###..###
###.#.#.###....
.####...#.#..#.

##.....
##.....
#..####
#..#.#.
#.#..#.
.#..##.
.#..##.
#.#..#.
#..#.#.
#..#.##
##.....

###.#.###......
.##..#.....#...
.#.#..#.#...###
##.###.#####...
#....#......###
#...####.#..#..
....###.###.#..
#.####.##.##.##
#.####..#.##.##
....###.###.#..
#...####.#..#..
#....#......###
##.###.#####...

#..##.#..####..#.
#...#.#..####..#.
..#.......##.....
.#####..........#
##..#............
.#.#.#..........#
##..##..........#
.##.#.##########.
.#.#....#....#...
##.##.###.##.###.
##.#.####....####
.#.#.##.######.##
###.#.#.######.#.
.#.#.#.########.#
##.##.#.##..##.#.

###.##.#..#.#
..####..###..
###...#..##.#
##.#..##.#.##
##.#..##.#.##
###...#..##.#
..####..###..
###.##.#..#.#
##.##.#.#..#.
..##..##.....
...#..##.....
........##..#
##...#.#.#..#
.##.##.#.#.#.
###...###..#.
###.##.##..#.
##.#....#.#..

#..#...####
#..##.#####
.#.....####
##..#.#####
#..#.##....
.#.####....
#.##.##....
.###.#.####
#..#.##....
##...##....
#...##.....
..##..##..#
..#........
#.##.##....
.##.#.#####

#.#...##.##.#
..#..####..##
#####....##..
#####....##..
.....####..##
#.#...##.##.#
#.#####......
.#.##.#######
##..#.#.####.
###.##..#..#.
....##..####.
##.##........
#..#..###..##
#.##..#..##..
#####..##..##

####.#.#.#..###
.#..#.##..#..#.
##.##.#.##....#
#..##.#.#.....#
#..###..#....#.
#..###..#....#.
#..##.#.#.....#
##.##.#.##....#
.#..#.##..#..#.
####.#.#.##.###
..#.#.#.##.#...
..#.#.#.##.#...
####.#.#.##.###
.#..#.##..#..#.
##.##.#.##....#

#.#...#.##..#.#
....###..####.#
.###.#.#####...
.###.#.#####...
....###..####.#
#.#...#.###.#.#
...#..#.#.#....
..###....#.#.#.
##..##.#....###
##..##.#....###
..###....#.#.#.
...#..#.#.#....
#.#...#.###.#.#
....###..####.#
.###.#.#####...

##.#.#...#..####.
..#.###...#......
...###....#.#..#.
..####..#.##....#
##..#....##.####.
##.#....#...#..#.
###.....##.##..##
##.####.#.###..##
..##..#...#.####.
......#.#.##....#
##.#..#..#.#.##.#
.....#.#...##..##
....###....#....#
#.#.###..##.####.
##.######...####.

.####..#.##
.#####.####
.#.###.####
.####..#.##
.#......#.#
..#...#..##
#..###.##..
##..#..####
#####.#.#..
###..#.####
.###.#....#
.###.#....#
###..#.####
#####.#.#..
##..#..####

...#######.##...#
..#.#..#.#.#####.
##..####..#...##.
##..####..#...##.
..#.#..#.#.#####.
...#######.###..#
..#.#...#..#..#.#
..#.##.....#.##..
##.##..##..##..##
.....#####...#.#.
..#.###.#.##.##..
..#...##.##..##.#
..#..#.##...###..

#..#.#.###...##
#.##...#.###..#
.#.############
.##.....#...###
##.....#...#.#.
...##....#.#..#
..#.....#.#.#..
..#.....#.#.#..
..###....#.#..#
##.....#...#.#.
.##.....#...###
.#.############
#.##...#.###..#
#..#.#.###...##
#..#.#.###...##

.#.##.#....####..
#......#.########
###..####...##...
#..###.###......#
.##..##....#..#..
.#....#.#.#....#.
#.####.#....##...

##.##..##..
##.##..##..
...#.######
##..##..##.
.#.....#...
.#.######.#
.##....##.#
.##....##.#
.#.######..
.#.....#...
##..##..##.
...#.######
##.##..##..

..##...#..#....#.
..#.....##.....#.
..#..##.##.##..#.
..#...##..##...#.
#####..#..#..####
..#..#......#..#.
...##........##..
####..##..##..###
..#..#.####.#..#.

...##....#.
.....#.#..#
....#...#.#
########...
..####.##..
.#####...##
###.###.##.
###.###.##.
.###.#...##
..####.##..
########...
....#...#.#
.....#.#..#
...##....#.
##..#.#####
.......##.#
.......##.#

.#...#...###.#.##
##...#####.#.##..
.####....##.###..
##.#..##.###..#..
#.##.##.##....##.
#.###..#...##..##
..##..#######..##
..##..#######..##
#.###..#...##..##
#.##.##.##....##.
##.#..##.###..#..
.####....##.####.
##...#####.#.##..
.#...#...###.#.##
.#...#...###.#.##
##...#####.#.##..
.####....##.####.

#####....##
##.##.##.##
..#.#.#..#.
..#..####..
..###.##.##
...#.####.#
####......#
##.#.#..#.#
####......#
###.######.
...#..##..#
##.........
..##..##..#
....#.##.#.
####.#..#.#

#.######.######
.##....##.####.
.#..##..#.####.
##..##..###..##
#..#..#..######
##..##..##....#
###....########
.#.####.#......
#.#....########
###....###....#
##.####.##....#

##########..#..
.######.####...
.#.##.#.###.###
.######...#.#..
#..##..#.#....#
##....##.##.###
#..##..#.###..#
#..##..#.#.###.
#..##..#.#.###.
#..##..#.###..#
##....##.##.###
#..##..#.#....#
.######...###..
.#.##.#.###.###
.######.####...

...#####.##.#####
...#####.##.#####
...#..#.#..#.#..#
#...#..#....#..#.
####.####..####.#
#.#....#.##.#....
##.#...######.#.#

#####..####
###.###....
.......####
....#######
..#.###....
.#.#..#....
.######....
#....#..##.
.#..###....
.###.##....
#####.#####
#.#....####
###.#.#....
..##.#.....
.#..#..####

..#..#.
#####.#
..#.##.
#####..
..###.#
......#
##..#..
###..##
######.
#####..
###..##

......#...##.#.
#####.##.####..
...#.#.####.##.
...#.#.####.##.
#####.##.#####.
......#...##.#.
##.#.#.##..#..#
...#..#.#....#.
#...##.#.##..#.
.#...#..#.#####
.....###.##.##.
.....###.##.##.
.#...#..#.#####

..#.##.#..#.#
##...........
##.##.#....#.
..#.....##...
...#.##....##
..###.#....#.
##.#..######.
....###...###
##.####.##.##
####..#....#.
.....#......#

.##.##.####.#
##..####..###
.#..###.##.##
##.#..#.##.#.
#.##.#.####.#
.##.....##...
####.........
.###.###..###
.#...#..##..#
..#....#..#..
###.#.#....#.
.#..####..###
##..####..###

.##....#.##
.##.#.#.###
#####.##.##
....###.#..
.##..#..#..
#.###..#...
#..#.###...
.##..#..###
######.....

###..##..##..
..##.###....#
#..#..#.#..#.
..###.#..##..
.#...##.#..#.
.#...##.#..#.
..###.#..##..
#..#..#.#..#.
..##.###....#
.##..##..##..
..##.........
..##.##.#..#.
##.#...######
....#########
....#...#..#.

.#.#..#.#...#
..##..##....#
..##..##....#
.#.#..#.#..##
##..##..###..
............#
#.##..##.##..
.#.#..#.#..#.
#..####..#.#.
..#....#...##
.#......#.##.

.#...#.##
#..#.##..
#..#..###
#..#...##
######.##
#####....
#..##.#..
.##...#..
######.##
.##....##
....#####
....##...
.##.##...
.......##
#######..
.##.#.#..
######.##

.####..##..##
.#..#.####.#.
.####.#..#.##
#....#.##.#..
#....#....#..
.####.####..#
..##........#
..##..#..#..#
.#..#......#.
#....######..
######.##.###

###..###..#....
.##..##.....##.
##.##.###.#.##.
........#......
#.#..#.#..#####
.#.##.#.#...##.
..####....#.##.
##....###.#....
..#..#...######
..#..#...#.....
..#..#..##..##.
...##...##.#..#
.######.#......
#########.####.
.#.##.#.##.####
##.##.##.......
##....##.##.##.

######..#
...#.###.
.#.....##
.#.....##
...#.##..
######..#
######..#

...#.#...#...
...#.#...#...
###.###.#.##.
....#.####.##
.....##..#..#
##..#..##..##
....##....##.
..##......###
##...#...#..#
#....#.##....
#....#.#.....

###..#.#..#
...#.##..##
##..#..##..
##.#....##.
###..#...##
..##..###..
..#.....#..
...##.##..#
..####..##.
.....#.##..
..###.#.#..
..#.#.#..##
..#.#.#..##
..###.#.#..
....##.##..

.....##..#.
#..#.#...##
....###..##
.....####..
.##.##.#.##
.....#.###.
.....#....#
#..##...###
####...####
####.#....#
######..###
#..##.##.##
#####..####
#####..#..#
#####..#.##

#.....###.#
#.###...#..
#.###...#..
#..#..###.#
##.##.#..#.
#####.#...#
.#.###.####
#.##.###...
..#..#..#..
.#...##.#..
.#...##.#..
..#..#..#..
#.##.###...
.#.###.####
#####.#...#
##.##.#..#.
#..#..###.#

...#..#.#..
##.##....##
..#..##.###
.....#.#.##
..##...####
##.....#..#
####..#.##.
...#.###...
##...#..###
###.###.#.#
##..##.##..
..#...#.#.#
..#..##..#.
..#.#..#...
..#.#..#...
..#...#..#.
..#...#.#.#

....#..#.#..##.
####.#.#....###
#..#.###.#...#.
.##.####.####..
.......#.....##
#..#..##..#.#..
####....##..#.#
....##....##.#.
####.#.#...####
#..#.#.####.###
#######.#.#####
####.#..###..##
####.#..###..##
#######.#.#####
#..#.#.########
####.#.#...####
....##....##.#.

.#..##....#.###
#..##.#..##.#..
##.##.#..##.#..
.#..##....#.###
###.###.##.#..#
.#..##.##..###.
##.#.##.####...
##.#.##.####...
.#..##.##..###.
###.###.##.#..#
.#..##....#.###
##.##.#..##.#..
#..##.#..##.#..
.#..##....#.###
#.#..###..###.#
#.###.#..#.##.#
.##..#.#...#..#

..#......
...#.....
..#.#..##
...###.##
...###.##
..#.#..##
...#.....
..#......
##.#####.
##...#..#
.#..#....
##.#...##
###.##.#.
...###..#
......#..
.....#..#
...##..#.

...#...#..##..#.#
......#..###...##
#.##.####.####.##
.######.#....#...
...###.##..#....#
...###.##..#.....
.######.#....#...
#.##.####.####.##
......#..###...##
...#...#..##..#.#
....#...##..###.#
..#.###.##......#
..#.###.##......#

###.####....#
...##.#.#..#.
###..#...##..
..#.#########
.###.#..#..#.
#.#..........
.#.##.#.####.
....###..##..
..#.#..#....#
..#.#..#....#
....###..##..
.#.##.#.####.
###..........

##..#...###..
.####.#..#.##
..##...#..###
..##....###..
........#####
##..####.....
#######......
#....###...##
#######.#####
#######.###..
..##..###..##
######.......
..##...###.##
.####.##...##
..##....##...
#.##.#.....##
.#..#.#.#####

#####.#.#
#.#.##...
#.#.##...
#####.#.#
##.######
###..##.#
...####..
..#.###..
..#.##...

....##.###.
####.......
#..#.....#.
....#.####.
#..####.###
#..##...#.#
#..##.#.#.#

####..###...###
######.....##..
######.#.#..#.#
######.##.#.##.
#####....#...#.
.....#.##.###..
....#..####.##.
#######.##.....
#####.##.#.#.#.
......#..#..#..
####.#..#.##...
....#....#....#
..........##...
####..###....#.
#..#####.#.....

.#.##...#......
#.#..#..#......
....#.#..######
.#..#.#..######
#.#..#..#......
.#.##...#......
..##.#.#.#.##.#
#...#..#.#....#
..#.#.#.##.##.#

######.#.##.#..
....####.#.#.#.
##.#.#..#.###..
..###.#....###.
..#..#.....##.#
##.#.###....#.#
..##.#.#.......
...##..####.##.
...####..#.##..
###.#####.###..
#########.###..
...####..#.##..
...##..####.##.
..##.#.#.......
##.#.###....#.#

.#.######...#.###
...#....#...##...
##.#....#.###.#..
#.#.####.#.#.####
#..........#...##
..#.####.#..###.#
#..........#.#...
#...#..#...#..##.
###..##..######.#
#.##....##.##.###
.##.####.##.#.#.#
.....##.....#..##
#..#.##.#..#.##.#
#..#.##.#..#.##.#
.....##.....#..##

##..#####.#.#
####...#.#.##
####...#.#.##
##..#######.#
..##..#######
##..#........
##.###....#.#
##....####.#.
..###...#####
###.#####.###
...#.####.#..
##.##...#...#
..#..##....##
...##...###.#
..##.#...#.#.
##.###.#..##.
####..##..#..

.##.###.#.##.#..#
.##...#..##.#####
####...#...#..##.
####.......######
#.##.##.###.#.##.
....##.####..####
......#..########
.....#.###..#.##.
.##.#..#######..#
.##......###..##.
.....###.##.##..#
####....#..#.#..#
####...#.#.#.....
#..#....#..######
....##.##.###.##.
#..###.#...##.##.
.##.#...#########

..####...#.#..#
########.......
#......##.##.##
##....##..#.##.
.#.##.#.....###
#.####.###.####
#.####.#####..#
##....###.#.##.
........###.###
#########...#..
#########......

########.####
#.#..#.#.....
...##....####
###..###.#..#
.#.##.#.#.##.
#......###.##
########.....
##.##.##..##.
###..###.####

#######...#....
.##..#..##.#..#
#..######.#####
#..#...#.##...#
.....#..##...##
.##.#....#..#.#
.....##.#....##
.##...#...#...#
.##...#...#...#
.....##.#....##
.##.#....#..#.#
.....#..##..###
#..#...#.##...#
#..######.#####
.##..#..##.#..#

.###...
##....#
##.#..#
.###...
...####
.#..#..
##....#
#####.#
.##..##
...##.#
.#.....
.#.....
...##.#
.##..##
#####.#
##....#
.#..#..

#..#....#..##
#..#....#..##
.#........#.#
.#.##..##.#..
.##.#..#.#..#
..#.#..#.#..#
#####..#####.
#.########.#.
.#.##..##.#.#

#.##..##.
....#....
....#....
#.#...##.
.##.###..
##..#..##
##..#..##

.##...##...
##.##....##
#..########
#..########
#####....##
.##...##...
....##..##.
..#..#..#..
#..########
#.#........
.###..##..#
#...######.
..#..####..

#..#...#.###...#.
#....###.#..#..##
..###.##.#...#...
..##.####..#..###
.#..###...##.####
#..####.##.##.#.#
##.####.##.##.#.#
.#..###...##.####
..##.####..#..###
..###.##.#...#...
#....###.#..#..##
#..#...#.###...#.
#..#...#.###...#.
#....###.#..#..##
..###.##.#...#...
..##.####..#..###
.#..###...##.####

##.....#.
##.....#.
.....##.#
##...#.##
#........
.#.....##
...##....
.##.#....
.#....#.#
.#....#.#
.##.#....
...##....
.#.....##
#........
##...#..#
.....##.#
##.....#.

##.#..##.#..#.##.
.#.##.#.#.##.#.#.
##.#####..##..###
#.#.####.####.###
###.#.#.#.##.#.#.
...###..#.##.#..#
#..#.###.#..#.###
#..#.###.#..#.###
...###..#.##.#..#
###.#.#.#.##.#.#.
..#.####.####.###

##..#......##
##.#.#.....#.
#.#####.##.##
#.#####.##.##
##.#.#.....#.
##..#......##
#..#...##..#.
#..##...##...
..###.#.#.##.
..###.#.#.##.
#..##...##...
#..#...##..#.
##..#.#....##
##.#.#.....#.
#.#####.##.##

.##..####..##.###
##.#..##..#.##.##
##.#......#.#####
.##..####..##.###
..#.##..##.#..#..
##.##.##..#.#####
#.##..##..##.#...

#.##..#..##....
.#.##.#.#######
...#...#.#..###
...##.####...##
#.##.#.##.#####
#.##.#.########
...##.####...##
...#...#.#..###
.#.##.#.#######
#.##..#..##....
##...####.###..

#.####..##..#
.....#.######
.#.#.#.#.####
##...##.#.##.
##.##...#.##.
..###..#.#..#
#####.#.##..#
..##..#..####
.##......#..#
.###.....#..#
..##..#..####

.##...#.####....#
#..#..#.#....###.
.#..####.##.#####
#..###.##..#.#..#
.##.#######..####
.##.#######..####
#..###.##..#.#..#

###.##.#..#.#.###
###.##.#..#.#.###
.##.##..###..##..
...#.####.##.##.#
####.#.#..#.##...
....####....#...#
..###.##.#.#...##

#####...##.##
#..##.##..##.
....##..#...#
#.#####....#.
....#....##..
#..#..#...#..
.##....######
.##.###..####
#..#.#...####
........##.##
.##.####..#..
.##..##.###.#
.##..##.###.#

...#.....##...#.#
...#.#.###....#..
..##.###......#.#
..##.###......#.#
...#.#.###....#..
...#.....##...#.#
#...#..##.#...#..
.####.######.....
###.##....#.....#
..##.#.###...#...
####..#...#..##..
..#.#####.###..##
##..#...#....#...
#######...###.#..
#######...###.#..
##..#...#....#...
..#######.###..##

####.#####.##.###
..#..###..#.#####
.#.#.####.##.#..#
#.####..#....#..#
.....#.#.#.###..#
##..#.####.#..##.
..#....#..####..#
.####.#####.#.##.
.####.#####.#.##.
..#....#..####..#
##..#.####.#..##.
.....#.#.#.###..#
#.####..#....#..#

#...####.
...#.##.#
.#..#..#.
#####..##
.####..##
.#..#..#.
...#.##.#
#...####.
#...#..#.
#.#.####.
#.#######

..##..#
###....
....##.
..##..#
....##.
...####
##..##.
###..#.
..#.##.
...#..#
..##..#
...#..#
##.####
####..#
..##..#

###....#...##..
..####..##....#
.####..#.######
####.#.#.#.##.#
#####..####..##
.###.####......
#.#..#.##.#..#.
#######..#.##.#
..#..###.##..##
.#.###.....##..
#...#....######
.###..#.#......
...##.##.#.##.#
...##.##.#.##.#
.####.#.#......
#...#....######
.#.###.....##..

#.#.###..#..#
.##.###.#....
#.###.#......
....##....##.
#.#.#..#.####
...#.#....##.
.#.##.#.#####
...##.#.#####
...#.#....##.

..#.##.#..##..#
#..#..######..#
##..#..###.#..#
###...##...#..#
##.########....
###...#.##.####
..#...##.#..##.

.##..##..##..
####..#..#..#
#..#..#..##.#
.##.#......#.
#######..####
####...##...#
.##....##....
#..#...##...#
.##.##....##.
####.#.##.#.#
....#..##..#.

.##.###
.##.###
.######
##...##
#..#.#.
.##....
.##....
#.#....
...###.
...###.
#.#.#..
.##....
.##....
#..#.#.
##...##"""

In [245]:
pactual = parse(actual)

In [246]:
sum(score(p) for p in pactual)

31265

# Part 2

In [240]:
def find_mirror2(pattern):
    for l_range, r_range in samples(pattern.shape[1]):
        
        l_side = pattern[:,l_range[0]:l_range[1]]
        r_side = np.flip(pattern[:, r_range[0]:r_range[1]], 1)
        if (l_side == r_side).sum() == ((l_side.shape[0] * l_side.shape[1]) - 1): # where exactly 1 false
            return l_range[1] # number of columns to left of mirror
    return None

find_mirror2(psample[1].T)

1

In [243]:
def score2(pattern):
    s = find_mirror2(pattern)
    if s is not None:
        return s
    else:
        return 100 * find_mirror2(pattern.T)

score2(psample[0]), score2(psample[1])

(300, 100)

In [247]:
sum(score2(p) for p in pactual)

39359